In [1]:
import torch
import torch.nn as nn
import torchvision
import numpy as np
import matplotlib.pyplot as plt
import tqdm

In [2]:
from google.colab import drive
drive.mount("content/")

Mounted at content/


In [3]:
dev = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [4]:
def evaluate(model, test_dl, crit):
  total = 0
  correct = 0
  tot_loss = 0
  model.eval()
  for x, y in test_dl:
    x = x.to(dev)
    y = y.to(dev)
    o = model(x)
    l = crit(o,y)
    tot_loss += l.item()
    correct += torch.sum(torch.argmax(o,axis=1) == y).item()
    total += len(y)
  test_loss = tot_loss / len(test_dl)
  test_acc = 100 * correct / total
  return test_loss, test_acc

def train_one_epoch(model, train_dl, crit, optim):
  total = 0
  correct = 0
  tot_loss = 0
  model.train()
  for x, y in train_dl:
    optim.zero_grad()
    x = x.to(dev)
    y = y.to(dev)
    o = model(x)
    l = crit(o,y)
    l.backward()
    optim.step()
    tot_loss += l.item()
    correct += torch.sum(torch.argmax(o,axis=1) == y).item()
    total += len(y)
  train_loss = tot_loss / len(train_dl)
  train_acc = 100 * correct / total
  return train_loss, train_acc

def train(model, train_dl, test_dl, crit, optim, lr_sched=None, epochs=10):
  for epoch in tqdm.tqdm_notebook(range(epochs)):
    train_loss, train_acc = train_one_epoch(model, train_dl, crit, optim)
    test_loss, test_acc = evaluate(model, test_dl, crit)
    if lr_sched is not None:
      lr_sched.step()
    print(f"epoch: {epoch}, train loss: {train_loss}, train accuracy: {train_acc:.2f}%, test loss: {test_loss}, test accuracy: {test_acc:.2f}%")

In [9]:
def get_activation_fn(activation):
  if activation == "relu":
    return nn.ReLU(inplace=True)
  elif activation == "sigmoid":
    return nn.Sigmoid()
  elif activation == "tanh":
    return nn.Tanh()
  elif activation == "selu":
    return nn.SELU(inplace=True)
  elif activation == "gelu":
    return nn.GELU()
  elif activation == "silu":
    return nn.SiLU(inplace=True)
  return None

def make_simple_fc_block(in_planes, out_planes, activation="relu"):
  return nn.Sequential(
      nn.Linear(in_planes, out_planes),
      get_activation_fn(activation)
  )

def make_simple_conv_block(in_planes, out_planes, kernel_size=3, stride=1, padding=0, activation="relu"):
  return nn.Sequential(
      nn.Conv2d(in_planes, out_planes, kernel_size, stride=stride, padding=padding, bias=False),
      nn.BatchNorm2d(out_planes),
      get_activation_fn(activation)
  )

In [6]:
class BasicResBlock(nn.Module):
  def __init__(self,in_planes, out_planes, kernel_size=3,stride=1):
    super().__init__()
    self.conv_block1 = nn.Sequential(
        nn.Conv2d(in_planes, out_planes, kernel_size,stride=stride,padding=1,bias=False),
        nn.BatchNorm2d(out_planes),
        nn.ReLU(inplace=True)
    )

    self.conv_block2 = nn.Sequential(
        nn.Conv2d(out_planes, out_planes, 1, bias=False),
        nn.BatchNorm2d(out_planes)
    )

    self.relu = nn.ReLU(inplace=True)

    self.shortcut = nn.Sequential()
    
    if in_planes != out_planes or stride != 1:
      self.shortcut = nn.Sequential(
          nn.Conv2d(in_planes, out_planes, 1, stride=stride),
          nn.BatchNorm2d(out_planes)
      )

  def forward(self, x):
    o = self.conv_block1(x)
    o = self.conv_block2(o)
    return self.relu(o + self.shortcut(x))

# MNIST

## MLP

In [7]:
train_ds = torchvision.datasets.MNIST("./content/MyDrive/MachineLearning/datasets/",train=True,transform=torchvision.transforms.ToTensor())
test_ds = torchvision.datasets.MNIST("./content/MyDrive/MachineLearning/datasets/",train=False,transform=torchvision.transforms.ToTensor())

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128)

In [ ]:
model = nn.Sequential(
    nn.Flatten(),
    nn.Linear(784,256),
    nn.ReLU(inplace=True),
    nn.Linear(256,10)
).to(dev)

In [ ]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.StepLR(optim, step_size=5,gamma=0.2)
train(model, train_dl, test_dl, crit, optim, lr_sched=lr_sched, epochs=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/15 [00:00<?, ?it/s]

epoch: 0, train loss: 0.5621210503648085, train accuracy: 85.87%, test loss: 0.31237425065681906, test accuracy: 90.95%
epoch: 1, train loss: 0.28638621719915475, train accuracy: 91.86%, test loss: 0.2474560518100669, test accuracy: 92.73%
epoch: 2, train loss: 0.23267405393567167, train accuracy: 93.39%, test loss: 0.2063493638924217, test accuracy: 93.92%
epoch: 3, train loss: 0.1960863298190428, train accuracy: 94.47%, test loss: 0.17713310603712557, test accuracy: 94.74%
epoch: 4, train loss: 0.1691688018709993, train accuracy: 95.28%, test loss: 0.1557395731429039, test accuracy: 95.19%
epoch: 5, train loss: 0.15055851459995642, train accuracy: 95.81%, test loss: 0.14805662940268086, test accuracy: 95.44%
epoch: 6, train loss: 0.1461392458496508, train accuracy: 95.94%, test loss: 0.14474938902414486, test accuracy: 95.59%
epoch: 7, train loss: 0.1423920891615056, train accuracy: 96.05%, test loss: 0.14167819291211758, test accuracy: 95.66%
epoch: 8, train loss: 0.1388560185459122

In [ ]:
torch.save(model.state_dict(), "./content/MyDrive/MachineLearning/models/mnist_mlp.ckpt")

## CNN

In [ ]:
model = nn.Sequential(
    make_simple_conv_block(1,16),
    nn.MaxPool2d(2),
    make_simple_conv_block(16,32),
    make_simple_conv_block(32,64),
    nn.MaxPool2d(2),
    make_simple_conv_block(64,128),
    nn.Flatten(),
    nn.Linear(512,10)
).to(dev)

In [ ]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.StepLR(optim, step_size=5,gamma=0.2)
train(model, train_dl, test_dl, crit, optim, lr_sched=lr_sched, epochs=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/15 [00:00<?, ?it/s]

epoch: 0, train loss: 0.1108574145768922, train accuracy: 97.07%, test loss: 0.008855714141500435, test accuracy: 98.29%
epoch: 1, train loss: 0.03557400322215917, train accuracy: 99.02%, test loss: 0.007129673647241536, test accuracy: 98.52%
epoch: 2, train loss: 0.023049822105017843, train accuracy: 99.40%, test loss: 0.005931062561865432, test accuracy: 98.86%
epoch: 3, train loss: 0.01572996737042155, train accuracy: 99.64%, test loss: 0.005162887140497395, test accuracy: 99.05%
epoch: 4, train loss: 0.010690011000677721, train accuracy: 99.82%, test loss: 0.005122179258161777, test accuracy: 99.06%
epoch: 5, train loss: 0.0069592445893627685, train accuracy: 99.90%, test loss: 0.003758824003017274, test accuracy: 99.30%
epoch: 6, train loss: 0.005691438357983438, train accuracy: 99.94%, test loss: 0.003735570904016229, test accuracy: 99.29%
epoch: 7, train loss: 0.005128165468235246, train accuracy: 99.96%, test loss: 0.003728539376644165, test accuracy: 99.32%
epoch: 8, train los

In [ ]:
torch.save(model.state_dict(), "./content/MyDrive/MachineLearning/models/mnist_cnn.ckpt")

## Using SELU

In [12]:
model = nn.Sequential(
    make_simple_conv_block(1,16,activation="selu"),
    nn.MaxPool2d(2),
    make_simple_conv_block(16,32,activation="selu"),
    make_simple_conv_block(32,64,activation="selu"),
    nn.MaxPool2d(2),
    make_simple_conv_block(64,128,activation="selu"),
    nn.Flatten(),
    nn.Linear(512,10)
).to(dev)

In [13]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.StepLR(optim, step_size=5,gamma=0.2)
train(model, train_dl, test_dl, crit, optim, lr_sched=lr_sched, epochs=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/15 [00:00<?, ?it/s]

epoch: 0, train loss: 0.11456031828889374, train accuracy: 96.56%, test loss: 0.07202562374870375, test accuracy: 97.79%
epoch: 1, train loss: 0.048205122094862524, train accuracy: 98.55%, test loss: 0.055409293080594606, test accuracy: 98.17%
epoch: 2, train loss: 0.033930732481946634, train accuracy: 99.03%, test loss: 0.050950450574775825, test accuracy: 98.24%
epoch: 3, train loss: 0.024903087760073574, train accuracy: 99.36%, test loss: 0.04858887927382713, test accuracy: 98.31%
epoch: 4, train loss: 0.01870573566075904, train accuracy: 99.57%, test loss: 0.04555462596743129, test accuracy: 98.42%
epoch: 5, train loss: 0.01160350892711607, train accuracy: 99.74%, test loss: 0.030488252271735995, test accuracy: 98.97%
epoch: 6, train loss: 0.009134000569367523, train accuracy: 99.84%, test loss: 0.03012592633060678, test accuracy: 99.02%
epoch: 7, train loss: 0.008002212627507648, train accuracy: 99.86%, test loss: 0.0300430966822723, test accuracy: 99.03%
epoch: 8, train loss: 0.0

# KMNIST

In [ ]:
train_ds = torchvision.datasets.KMNIST("./content/MyDrive/MachineLearning/datasets/",train=True,transform=torchvision.transforms.ToTensor())
test_ds = torchvision.datasets.KMNIST("./content/MyDrive/MachineLearning/datasets/",train=False,transform=torchvision.transforms.ToTensor())

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128)

In [ ]:
model = nn.Sequential(
    make_simple_conv_block(1,16),
    nn.MaxPool2d(2),
    make_simple_conv_block(16,32),
    make_simple_conv_block(32,64),
    nn.MaxPool2d(2),
    make_simple_conv_block(64,128),
    nn.Flatten(),
    nn.Linear(512,10)
).to(dev)

In [ ]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.StepLR(optim, step_size=5,gamma=0.2)
train(model, train_dl, test_dl, crit, optim, lr_sched=lr_sched, epochs=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/15 [00:00<?, ?it/s]

epoch: 0, train loss: 0.24297877096894707, train accuracy: 93.25%, test loss: 0.05477526548828906, test accuracy: 90.16%
epoch: 1, train loss: 0.0757294697409023, train accuracy: 97.95%, test loss: 0.03776039747096328, test accuracy: 93.51%
epoch: 2, train loss: 0.044771832491416155, train accuracy: 98.89%, test loss: 0.03405599299270207, test accuracy: 94.31%
epoch: 3, train loss: 0.02830034546427, train accuracy: 99.41%, test loss: 0.03367416039585813, test accuracy: 94.50%
epoch: 4, train loss: 0.01771238098578301, train accuracy: 99.74%, test loss: 0.033034831094843495, test accuracy: 94.64%
epoch: 5, train loss: 0.010324730287129278, train accuracy: 99.88%, test loss: 0.029444747213234524, test accuracy: 95.25%
epoch: 6, train loss: 0.008363067401823267, train accuracy: 99.94%, test loss: 0.029503875560979092, test accuracy: 95.29%
epoch: 7, train loss: 0.007420097550959078, train accuracy: 99.95%, test loss: 0.029628313549641353, test accuracy: 95.31%
epoch: 8, train loss: 0.0067

In [ ]:
torch.save(model.state_dict(), "./content/MyDrive/MachineLearning/models/kmnist_cnn.ckpt")

# FashionMNIST

In [ ]:
train_ds = torchvision.datasets.FashionMNIST("./content/MyDrive/MachineLearning/datasets/",train=True,transform=torchvision.transforms.ToTensor())
test_ds = torchvision.datasets.FashionMNIST("./content/MyDrive/MachineLearning/datasets/",train=False,transform=torchvision.transforms.ToTensor())

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128)

In [ ]:
model = nn.Sequential(
    make_simple_conv_block(1,16),
    nn.MaxPool2d(2),
    make_simple_conv_block(16,32),
    make_simple_conv_block(32,64),
    nn.MaxPool2d(2),
    make_simple_conv_block(64,128),
    nn.Flatten(),
    nn.Linear(512,10)
).to(dev)

In [ ]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.StepLR(optim, step_size=5,gamma=0.2)
train(model, train_dl, test_dl, crit, optim, lr_sched=lr_sched, epochs=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/15 [00:00<?, ?it/s]

epoch: 0, train loss: 0.42451695611736157, train accuracy: 84.72%, test loss: 0.058283795298798, test accuracy: 87.31%
epoch: 1, train loss: 0.2830337393385515, train accuracy: 89.69%, test loss: 0.052338694839843554, test accuracy: 88.60%
epoch: 2, train loss: 0.23821220699467385, train accuracy: 91.45%, test loss: 0.049772873234901344, test accuracy: 89.43%
epoch: 3, train loss: 0.20688534502599285, train accuracy: 92.63%, test loss: 0.04659483656446054, test accuracy: 90.31%
epoch: 4, train loss: 0.18157475992941907, train accuracy: 93.53%, test loss: 0.04740158147585671, test accuracy: 90.19%
epoch: 5, train loss: 0.13228494021048678, train accuracy: 95.42%, test loss: 0.04176310138471091, test accuracy: 91.39%
epoch: 6, train loss: 0.11796480794546446, train accuracy: 96.06%, test loss: 0.041807505049939346, test accuracy: 91.51%
epoch: 7, train loss: 0.10886655220471973, train accuracy: 96.48%, test loss: 0.042125865531120216, test accuracy: 91.57%
epoch: 8, train loss: 0.1011311

In [ ]:
torch.save(model.state_dict(), "./content/MyDrive/MachineLearning/models/fashionmnist_cnn.ckpt")

# CIFAR10

## CNN

In [14]:
train_ds = torchvision.datasets.CIFAR10("./content/MyDrive/MachineLearning/datasets/",train=True,transform=torchvision.transforms.ToTensor())
test_ds = torchvision.datasets.CIFAR10("./content/MyDrive/MachineLearning/datasets/",train=False,transform=torchvision.transforms.ToTensor())

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128)

In [15]:
model = nn.Sequential(
    make_simple_conv_block(3,16),
    nn.MaxPool2d(2),
    make_simple_conv_block(16,32),
    nn.MaxPool2d(2),
    make_simple_conv_block(32,64),
    make_simple_conv_block(64,128),
    nn.Flatten(),
    nn.Linear(512,10)
).to(dev)

In [16]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.StepLR(optim, step_size=5,gamma=0.2)
train(model, train_dl, test_dl, crit, optim, lr_sched=lr_sched, epochs=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/15 [00:00<?, ?it/s]

epoch: 0, train loss: 1.366252053576662, train accuracy: 50.52%, test loss: 1.1968834792511374, test accuracy: 57.58%
epoch: 1, train loss: 1.0080433852227448, train accuracy: 64.24%, test loss: 1.4189497851118256, test accuracy: 52.61%
epoch: 2, train loss: 0.8525633964392231, train accuracy: 69.93%, test loss: 1.5538100819044476, test accuracy: 53.20%
epoch: 3, train loss: 0.7499770878068627, train accuracy: 73.74%, test loss: 1.2534322135056122, test accuracy: 60.72%
epoch: 4, train loss: 0.6691456378420906, train accuracy: 76.76%, test loss: 1.1268335837352126, test accuracy: 63.19%
epoch: 5, train loss: 0.5184702159041334, train accuracy: 82.74%, test loss: 0.7880913818938823, test accuracy: 73.44%
epoch: 6, train loss: 0.4775528361272934, train accuracy: 84.38%, test loss: 0.7875127965890909, test accuracy: 73.42%
epoch: 7, train loss: 0.4523991384088536, train accuracy: 85.28%, test loss: 0.7886465463457228, test accuracy: 73.64%
epoch: 8, train loss: 0.4305694352101792, train a

In [ ]:
torch.save(model.state_dict(), "./content/MyDrive/MachineLearning/models/cifar10_cnn.ckpt")

## Using SELU

In [17]:
model = nn.Sequential(
    make_simple_conv_block(3,16,activation="selu"),
    nn.MaxPool2d(2),
    make_simple_conv_block(16,32,activation="selu"),
    nn.MaxPool2d(2),
    make_simple_conv_block(32,64,activation="selu"),
    make_simple_conv_block(64,128,activation="selu"),
    nn.Flatten(),
    nn.Linear(512,10)
).to(dev)

In [18]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.StepLR(optim, step_size=5,gamma=0.2)
train(model, train_dl, test_dl, crit, optim, lr_sched=lr_sched, epochs=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/15 [00:00<?, ?it/s]

epoch: 0, train loss: 1.4855990318386145, train accuracy: 47.48%, test loss: 1.7421744729899153, test accuracy: 43.40%
epoch: 1, train loss: 1.190734077292635, train accuracy: 57.93%, test loss: 1.5771581221230422, test accuracy: 50.93%
epoch: 2, train loss: 1.075207787401536, train accuracy: 62.34%, test loss: 1.5508233218253413, test accuracy: 51.82%
epoch: 3, train loss: 0.9941161738332275, train accuracy: 65.40%, test loss: 1.2185806594317472, test accuracy: 58.89%
epoch: 4, train loss: 0.931999277428288, train accuracy: 67.61%, test loss: 1.1946217025382608, test accuracy: 58.95%
epoch: 5, train loss: 0.76810926656284, train accuracy: 73.57%, test loss: 0.8865986096708081, test accuracy: 69.65%
epoch: 6, train loss: 0.7364221730500536, train accuracy: 74.63%, test loss: 0.8818524238429492, test accuracy: 69.79%
epoch: 7, train loss: 0.7177190748626924, train accuracy: 75.29%, test loss: 0.8802379452729527, test accuracy: 69.95%
epoch: 8, train loss: 0.7013590267247252, train accur

## ResNet

In [19]:
model = nn.Sequential(
    BasicResBlock(3,16),
    nn.MaxPool2d(2),
    BasicResBlock(16,32),
    nn.MaxPool2d(2),
    BasicResBlock(32,64),
    nn.MaxPool2d(2),
    BasicResBlock(64,128),
    nn.MaxPool2d(2),
    BasicResBlock(128,256),
    nn.Flatten(),
    nn.Linear(1024,10)
).to(dev)

In [20]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.StepLR(optim, step_size=5,gamma=0.2)
train(model, train_dl, test_dl, crit, optim, lr_sched=lr_sched, epochs=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/15 [00:00<?, ?it/s]

epoch: 0, train loss: 1.394713265054366, train accuracy: 50.58%, test loss: 1.313536070570161, test accuracy: 53.73%
epoch: 1, train loss: 0.9179353838991327, train accuracy: 67.30%, test loss: 1.0133675019952315, test accuracy: 64.89%
epoch: 2, train loss: 0.7372271287471742, train accuracy: 73.80%, test loss: 0.9569090944302233, test accuracy: 67.86%
epoch: 3, train loss: 0.616171497975469, train accuracy: 78.38%, test loss: 0.9121798805043667, test accuracy: 70.14%
epoch: 4, train loss: 0.5170112611997463, train accuracy: 81.86%, test loss: 0.8631513880777962, test accuracy: 71.65%
epoch: 5, train loss: 0.32251059753663097, train accuracy: 89.17%, test loss: 0.7642864358576038, test accuracy: 76.02%
epoch: 6, train loss: 0.2459491514565085, train accuracy: 92.19%, test loss: 0.791411135392853, test accuracy: 76.01%
epoch: 7, train loss: 0.19164226176526844, train accuracy: 94.48%, test loss: 0.8249155679081059, test accuracy: 75.85%
epoch: 8, train loss: 0.14509354357409965, train a

In [ ]:
torch.save(model.state_dict(), "./content/MyDrive/MachineLearning/models/cifar10_resnet.ckpt")

# CIFAR100

In [ ]:
train_ds = torchvision.datasets.CIFAR100("./content/MyDrive/MachineLearning/datasets/",train=True,transform=torchvision.transforms.ToTensor())
test_ds = torchvision.datasets.CIFAR100("./content/MyDrive/MachineLearning/datasets/",train=False,transform=torchvision.transforms.ToTensor())

train_dl = torch.utils.data.DataLoader(train_ds, batch_size=128)
test_dl = torch.utils.data.DataLoader(test_ds, batch_size=128)

In [ ]:
model = nn.Sequential(
    BasicResBlock(3,16),
    nn.MaxPool2d(2),
    BasicResBlock(16,32),
    nn.MaxPool2d(2),
    BasicResBlock(32,64),
    nn.MaxPool2d(2),
    BasicResBlock(64,128),
    nn.MaxPool2d(2),
    BasicResBlock(128,256),
    nn.Flatten(),
    nn.Linear(1024,100)
).to(dev)

In [ ]:
crit = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1)
lr_sched = torch.optim.lr_scheduler.StepLR(optim, step_size=5,gamma=0.2)
train(model, train_dl, test_dl, crit, optim, lr_sched=lr_sched, epochs=15)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:39: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/15 [00:00<?, ?it/s]

epoch: 0, train loss: 3.3707260170860973, train accuracy: 19.00%, test loss: 0.6582874251753473, test accuracy: 20.73%
epoch: 1, train loss: 2.5855386799863536, train accuracy: 33.52%, test loss: 0.5888009754288227, test accuracy: 27.86%
epoch: 2, train loss: 2.194731657767235, train accuracy: 41.98%, test loss: 0.5252361169556523, test accuracy: 34.25%
epoch: 3, train loss: 1.9015723344919933, train accuracy: 48.85%, test loss: 0.5223304469262242, test accuracy: 34.98%
epoch: 4, train loss: 1.6546543144509005, train accuracy: 54.96%, test loss: 0.4967417918202822, test accuracy: 37.93%
epoch: 5, train loss: 1.20185375800523, train accuracy: 67.78%, test loss: 0.4241513821779919, test accuracy: 45.13%
epoch: 6, train loss: 1.0484590595945373, train accuracy: 72.53%, test loss: 0.42633641132003514, test accuracy: 45.32%
epoch: 7, train loss: 0.9437271028833316, train accuracy: 75.86%, test loss: 0.429403008402461, test accuracy: 45.48%
epoch: 8, train loss: 0.8513106551316693, train acc

In [ ]:
torch.save(model.state_dict(), "./content/MyDrive/MachineLearning/models/cifar100_resnet.ckpt")